In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2

## Download ARKit data

Use ARKitscenes downloading script to download lowres depth, image, and confidence frames. Please refer to ARKitscenes repo for more details: https://github.com/apple/ARKitScenes/blob/main/DATA.md

Example:

`python3 download_data.py raw --video_id 47333462 --raw_dataset_assets lowres_depth confidence vga_wide --split Training --download_dir ./`

## Pick video clip and convert to right format

In [ ]:
## ARKit raw data folders
depth_dir = '/home/szhbchr/ARKitScenes/raw/Training/47333462/lowres_depth'
color_dir = '/home/szhbchr/ARKitScenes/raw/Training/47333462/vga_wide'
conf_dir = '/home/szhbchr/ARKitScenes/raw/Training/47333462/confidence'

files_depth = sorted(os.listdir(depth_dir))
files_color = sorted(os.listdir(color_dir))
files_conf = sorted(os.listdir(conf_dir))

## Processed input file folders
depth_save = 'data/arkit_3_long/depth'
color_save = 'data/arkit_3_long/color'
conf_save = 'data/arkit_3_long/conf'

if not os.path.exists(depth_save):
    os.mkdir(depth_save)
    os.mkdir(color_save)
    os.mkdir(conf_save)

## Video clip from frame 170-270, stride = 2
for ii, idx in enumerate(np.arange(170, 270, 2)):
    file_color = files_color[idx]
    file_depth = files_depth[idx*2]
    file_conf = files_conf[idx*2]
    # print(file_color == file_depth)
    img = imageio.imread(os.path.join(color_dir, file_color))
    imageio.imwrite(os.path.join(color_save, '{:06d}.png'.format(ii)), img)
    d = cv2.resize(imageio.imread(os.path.join(depth_dir, file_depth)), \
                   None, fx = 2.5, fy = 2.5, interpolation = cv2.INTER_AREA)
    d = d.astype(np.float32)/256/17*10
    np.save(os.path.join(depth_save, '{:06d}.npy'.format(ii)), d)
    conf = cv2.resize(imageio.imread(os.path.join(conf_dir, file_conf)).astype(np.float32)/2.0, \
                   None, fx = 2.5, fy = 2.5, interpolation = cv2.INTER_AREA)
    np.save(os.path.join(conf_save, '{:06d}.npy'.format(ii)), conf)

## Run evalution code (downsample ARKit depth with confidence)

In [ ]:
!bash demo_arkit_conf.sh

## Visualize result

In [ ]:
pred = np.load('results/demo_arkit_3/{:08d}.npy'.format(20))[0,0]
plt.imshow(pred, vmin = 0.0, vmax = 1.0)